# Training a MLP using backpropagation

* **part 1** we train using `MLP`. 
* **part 2** we train using `Sequential` and `Linear`. 

In [18]:
import jax.numpy as jnp
import jax
from net import MLP, Linear, Sequential
from module import Module
from function import ReLU, Tanh, LogSoftmax
from jax import grad, jit

## Part 1
We first define useful parameters that we are going to use later. Note that our model is a MLP imported from `net.py`

In [25]:
# hyperparameters
layer_sizes = [784, 512, 256, 10]
step_size = 0.01
num_epochs = 5
batch_size = 128
n_targets = 10
key = jax.random.PRNGKey(0)
function = ReLU()
log_softmax = LogSoftmax()

# to create a model is as simple as that
model = MLP(layer_sizes, function, key)
params = model.params # we need to extract parameters

Helper functions, trainloader and other stuff...

In [26]:
# some helper functions
def one_hot(x, k, dtype=jnp.float32):
  """Create a one-hot encoding of x of size k."""
  return jnp.array(x[:, None] == jnp.arange(k), dtype)
  
def accuracy(params, images, targets):
  target_class = jnp.argmax(targets, axis=1)
  predicted_class = jnp.argmax(model(images,params), axis=1)
  return jnp.mean(predicted_class == target_class)

def loss(params, images, targets):
  preds = model(images, params)
  preds = log_softmax(preds)
  return -jnp.mean(preds * targets)

@jit
def update(params, x, y):
  grads = grad(loss)(params, x, y)
  return [(w - step_size * dw, b - step_size * db)
          for (w, b), (dw, db) in zip(params, grads)]


import numpy as np
from torch.utils import data
from torchvision.datasets import MNIST

def numpy_collate(batch):
  if isinstance(batch[0], np.ndarray):
    return np.stack(batch)
  elif isinstance(batch[0], (tuple,list)):
    transposed = zip(*batch)
    return [numpy_collate(samples) for samples in transposed]
  else:
    return np.array(batch)

class NumpyLoader(data.DataLoader):
  def __init__(self, dataset, batch_size=1,
                shuffle=False, sampler=None,
                batch_sampler=None, num_workers=0,
                pin_memory=False, drop_last=False,
                timeout=0, worker_init_fn=None):
    super(self.__class__, self).__init__(dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        sampler=sampler,
        batch_sampler=batch_sampler,
        num_workers=num_workers,
        collate_fn=numpy_collate,
        pin_memory=pin_memory,
        drop_last=drop_last,
        timeout=timeout,
        worker_init_fn=worker_init_fn)

class FlattenAndCast(object):
  def __call__(self, pic):
    return np.ravel(np.array(pic, dtype=jnp.float32))

# Define our dataset, using torch datasets
mnist_dataset = MNIST('/tmp/mnist/', download=True, transform=FlattenAndCast())
training_generator = NumpyLoader(mnist_dataset, batch_size=batch_size, num_workers=0)

# Get the full train dataset (for checking accuracy while training)
train_images = np.array(mnist_dataset.train_data).reshape(len(mnist_dataset.train_data), -1)
train_labels = one_hot(np.array(mnist_dataset.train_labels), n_targets)

# Get full test dataset
mnist_dataset_test = MNIST('/tmp/mnist/', download=True, train=False)
test_images = jnp.array(mnist_dataset_test.test_data.numpy().reshape(len(mnist_dataset_test.test_data), -1), dtype=jnp.float32)
test_labels = one_hot(np.array(mnist_dataset_test.test_labels), n_targets)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Here we train using backpropagation

In [27]:
import time

for epoch in range(num_epochs):
  start_time = time.time()
  for x, y in training_generator:
    y = one_hot(y, n_targets)
    params = update(params, x, y)
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_images, train_labels)
  test_acc = accuracy(params, test_images, test_labels)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

Epoch 0 in 4.93 sec
Training set accuracy 0.9045000076293945
Test set accuracy 0.911899983882904
Epoch 1 in 4.24 sec
Training set accuracy 0.9273999929428101
Test set accuracy 0.9307000041007996
Epoch 2 in 4.22 sec
Training set accuracy 0.9414499998092651
Test set accuracy 0.939799964427948
Epoch 3 in 4.21 sec
Training set accuracy 0.9497500061988831
Test set accuracy 0.9477999806404114
Epoch 4 in 4.26 sec
Training set accuracy 0.9563000202178955
Test set accuracy 0.9527999758720398


## Part 1
We first define useful parameters that we are going to use later. Note that our model is a costum model built using `Sequential` and `Linear`.

In [28]:
class myModel(Module):
    def __init__(self):
        self.layers = Sequential([
            Linear(28*28,512,key),
            function,
            Linear(512,256,key),
            function,
            Linear(256,10,key)
        ])
        self.params = self.layers.generate_parameters()

    def forward(self, data, params):
        return self.layers(data, params)

# we define the model as simple as that
model = myModel()
params = model.params

Let's train

In [29]:
for epoch in range(num_epochs):
  start_time = time.time()
  for x, y in training_generator:
    y = one_hot(y, n_targets)
    params = update(params, x, y)
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_images, train_labels)
  test_acc = accuracy(params, test_images, test_labels)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

Epoch 0 in 5.10 sec
Training set accuracy 0.9095166921615601
Test set accuracy 0.9138000011444092
Epoch 1 in 4.17 sec
Training set accuracy 0.9317666888237
Test set accuracy 0.9315999746322632
Epoch 2 in 4.85 sec
Training set accuracy 0.9434833526611328
Test set accuracy 0.9411999583244324
Epoch 3 in 4.12 sec
Training set accuracy 0.9513999819755554
Test set accuracy 0.9483999609947205
Epoch 4 in 4.18 sec
Training set accuracy 0.9579166769981384
Test set accuracy 0.9537999629974365
